# Traffic Sign Classification with Keras

Keras exists to make coding deep neural networks simpler. To demonstrate just how easy it is, you’re going to use Keras to build a convolutional neural network in a few dozen lines of code.

You’ll be connecting the concepts from the previous lessons to the methods that Keras provides.

## Dataset

The network you'll build with Keras is similar to the example that you can find in Keras’s GitHub repository that builds out a [convolutional neural network for MNIST](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py). 

However, instead of using the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, you're going to use the [German Traffic Sign Recognition Benchmark](http://benchmark.ini.rub.de/?section=gtsrb&subsection=news) dataset that you've used previously.

You can download pickle files with sanitized traffic sign data here.

## Overview

Here are the steps you'll take to build the network:

1. First load the data.
2. Build a feedforward neural network to classify traffic signs.
3. Build a convolutional neural network to classify traffic signs.

Keep an eye on the network’s accuracy over time. Once the accuracy reaches the 98% range, you can be confident that you’ve built and trained an effective model.

## Load the Data

Start by importing the data from the pickle file.

In [1]:
# TODO: Implement load the data here.
import keras

# Load pickled data
import pickle

# TODO: fill this in based on where you saved the training and testing data
training_file = 'train.p'
testing_file = 'test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(X_train.shape[0] == y_train.shape[0]), "The number of images is not equal to the number of labels."
assert(X_train.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."

Using TensorFlow backend.


## Normalize the data

Now that you've loaded the training data, normalize the input so that it has a mean of 0 and a range between -0.5 and 0.5.

In [2]:
# TODO: Implement data normalization here.

import random
import numpy as np

def normalize_greyscale(image_data):
    x_min = np.min(image_data)
    x_max = np.max(image_data)
    a     = -0.5
    b     = 0.5
    return a + np.divide( (image_data - x_min ) * ( b - a), x_max - x_min) 

def preprocess_all_images(X_train, X_test, img_size):
    
    # Preallocate space for all images, for just a few tens of thousands of 
    # small images this should not need advanced caching techniques
    n_train = X_train.shape[0]
    n_test  = X_test.shape[0]
    
    train_features = np.zeros( [n_train, img_size, img_size, 3] )
    test_features  = np.zeros( [n_test,  img_size, img_size, 3] )

    for i in range(n_train):
        train_features[i,] = normalize_greyscale(X_train[i])
    for i in range(n_test):
        test_features[i,]  = normalize_greyscale( X_test[i] )
    
    return train_features, test_features

X_train, X_test = preprocess_all_images(X_train, X_test, 32)

# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(round(np.mean(X_train)) == 0), "The mean of the input data is: %f" % np.mean(X_train)
assert(np.min(X_train) == -0.5 and np.max(X_train) == 0.5), "The range of the input data is: %.1f to %.1f" % (np.min(X_train), np.max(X_train))

## Build a Two-Layer Feedfoward Network

The code you've written so far is for data processing, not specific to Keras. Here you're going to build Keras-specific code.

Build a two-layer feedforward neural network, with 128 neurons in the fully-connected hidden layer. 

To get started, review the Keras documentation about [models](https://keras.io/models/sequential/) and [layers](https://keras.io/layers/core/).

The Keras example of a [Multi-Layer Perceptron](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py) network is similar to what you need to do here. Use that as a guide, but keep in mind that there are a number of differences.

In [3]:
# TODO: Build a two-layer feedforward neural network with Keras here.
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils

batch_size = 128
nb_classes = 43
nb_epoch   = 2

n_train = X_train.shape[0]
n_test  = X_test.shape[0]

X_train = X_train.reshape(n_train, 32*32*3)
X_test  = X_test.reshape(n_test, 32*32*3)
X_train = X_train.astype('float32')
X_test  = X_test.astype('float32')

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test  = np_utils.to_categorical(y_test, nb_classes)

model = Sequential()
# 1 neuron per pixel, really Udacity?
model.add(Dense(32*32*3, input_shape=(32*32*3,), name='input'))
model.add(Activation('relu'))
model.add(Dense(128, name='hidden1'))
model.add(Activation('relu'))
model.add(Dense(43, name='output'))
model.add(Activation('softmax'))

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(model.get_layer(name="hidden1").input_shape == (None, 32*32*3)), "The input shape is: %s" % model.get_layer(name="hidden1").input_shape
assert(model.get_layer(name="output").output_shape == (None, 43)), "The output shape is: %s" % model.get_layer(name="output").output_shape 

## Train the Network
Compile and train the network for 2 epochs. [Use the `adam` optimizer, with `categorical_crossentropy` loss.](https://keras.io/models/sequential/)

Hint 1: In order to use categorical cross entropy, you will need to [one-hot encode the labels](https://github.com/fchollet/keras/blob/master/keras/utils/np_utils.py).

Hint 2: In order to pass the input images to the fully-connected hidden layer, you will need to [reshape the input](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py).

Hint 3: Keras's `.fit()` method returns a `History.history` object, which the tests below use. Save that to a variable named `history`.

In [5]:
# TODO: Compile and train the model here.
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(X_train, Y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1)
score = model.evaluate(X_test, Y_test, verbose=0)

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['acc'][0] > 0.5), "The training accuracy was: %.3f" % history.history['acc']

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input (Dense)                    (None, 3072)          9440256     dense_input_1[0][0]              
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 3072)          0           input[0][0]                      
____________________________________________________________________________________________________
hidden1 (Dense)                  (None, 128)           393344      activation_1[0][0]               
____________________________________________________________________________________________________
activation_2 (Activation)        (None, 128)           0           hidden1[0][0]                    
___________________________________________________________________________________________

## Validate the Network
Split the training data into a training and validation set.

Measure the [validation accuracy](https://keras.io/models/sequential/) of the network after two training epochs.

Hint: [Use the `train_test_split()` method](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) from scikit-learn.

In [8]:
# TODO: Split some of the training data into a validation dataset.
from sklearn.model_selection import train_test_split

X_train, X_val, Y_train, Y_val = train_test_split(
    X_train,
    Y_train,
    test_size=0.25,
    random_state=1)

# TODO: Compile and train the model to measure validation accuracy.
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(X_train, Y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_val, Y_val))
score = model.evaluate(X_test, Y_test, verbose=0)

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(round(X_train.shape[0] / float(X_val.shape[0])) == 3), "The training set is %.3f times larger than the validation set." % X_train.shape[0] / float(X_val.shape[0])
assert(history.history['val_acc'][0] > 0.6), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

Train on 19702 samples, validate on 6568 samples
Epoch 1/2
19702/19702 [==============================] - 39s - loss: 0.3336 - acc: 0.9224 - val_loss: 0.1690 - val_acc: 0.9514
Epoch 2/2
19702/19702 [==============================] - 38s - loss: 0.2457 - acc: 0.9364 - val_loss: 0.2659 - val_acc: 0.9335


**Validation Accuracy**: (0.9335)

## Congratulations
You've built a feedforward neural network in Keras!

Don't stop here! Next, you'll add a convolutional layer to drive.py.

## Convolutions
Build a new network, similar to your existing network. Before the hidden layer, add a 3x3 [convolutional layer](https://keras.io/layers/convolutional/#convolution2d) with 32 filters and valid padding.

Then compile and train the network.

Hint 1: The Keras example of a [convolutional neural network](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py) for MNIST would be a good example to review.

Hint 2: Now that the first layer of the network is a convolutional layer, you no longer need to reshape the input images before passing them to the network. You might need to reload your training data to recover the original shape.

Hint 3: Add a [`Flatten()` layer](https://keras.io/layers/core/#flatten) between the convolutional layer and the fully-connected hidden layer.

In [14]:
# TODO: Re-construct the network and add a convolutional layer before the first fully-connected layer.
# TODO: Compile and train the model.
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D

nb_epoch   = 1

# input image dimensions
img_rows, img_cols = 32, 32
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)

n_train = X_train.shape[0]
n_test  = X_test.shape[0]
n_val  = X_val.shape[0]

X_train = X_train.reshape(n_train, 32, 32, 3)
X_test  = X_test.reshape(n_test, 32, 32, 3)
X_val   = X_val.reshape(n_val, 32, 32, 3)
X_train = X_train.astype('float32')
X_test  = X_test.astype('float32')

input_shape = (32, 32, 3)

model = Sequential()
# 1 neuron per pixel, really Udacity?
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(43, name='output'))
model.add(Activation('softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

history = model.fit(X_train, Y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_val, Y_val))
score = model.evaluate(X_test, Y_test, verbose=0)

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][0] > 0.7), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_9 (Convolution2D)  (None, 30, 30, 32)    896         convolution2d_input_5[0][0]      
____________________________________________________________________________________________________
activation_20 (Activation)       (None, 30, 30, 32)    0           convolution2d_9[0][0]            
____________________________________________________________________________________________________
convolution2d_10 (Convolution2D) (None, 28, 28, 32)    9248        activation_20[0][0]              
____________________________________________________________________________________________________
activation_21 (Activation)       (None, 28, 28, 32)    0           convolution2d_10[0][0]           
___________________________________________________________________________________________

**Validation Accuracy**: (fill in here)

## Pooling
Re-construct your network and add a 2x2 [pooling layer](https://keras.io/layers/pooling/#maxpooling2d) immediately following your convolutional layer.

Then compile and train the network.

In [ ]:
# TODO: Re-construct the network and add a pooling layer after the convolutional layer.
# TODO: Compile and train the model.

"""Done above"""

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][0] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

**Validation Accuracy**: (fill in here)

## Dropout
Re-construct your network and add [dropout](https://keras.io/layers/core/#dropout) after the pooling layer. Set the dropout rate to 50%.

In [ ]:
# TODO: Re-construct the network and add dropout after the pooling layer.
# TODO: Compile and train the model.

"""Done above"""

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][0] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

In [ ]:
**Validation Accuracy**: (fill in here)

## Optimization
Congratulations! You've built a neural network with convolutions, pooling, dropout, and fully-connected layers, all in just a few lines of code.

Have fun with the model and see how well you can do! Add more layers, or regularization, or different padding, or batches, or more training epochs.

What is the best validation accuracy you can achieve?

In [ ]:
"""This is not important now"""

**Best Validation Accuracy:** (fill in here)

## Testing
Once you've picked out your best model, it's time to test it.

Load up the test data and use the [`evaluate()` method](https://keras.io/models/model/#evaluate) to see how well it does.

Hint 1: After you load your test data, don't forget to normalize the input and one-hot encode the output, so it matches the training data.

Hint 2: The `evaluate()` method should return an array of numbers. Use the `metrics_names()` method to get the labels.

In [15]:
with open('./test.p', mode='rb') as f:
    test = pickle.load(f)
    
X_test = test['features']
y_test = test['labels']
X_test = X_test.astype('float32')
X_test /= 255
X_test -= 0.5
Y_test = np_utils.to_categorical(y_test, 43)

model.evaluate(X_test, Y_test)

12630/12630 [==============================] - 13s    


[1.5968633237945968, 0.59786223277909734]

**Test Accuracy:** (fill in here)

## Summary
Keras is a great tool to use if you want to quickly build a neural network and evaluate performance.